#

In [9]:
def sub_orbits(G): ##need a fix (G is not the automorphism of the resulting graphs)
	S = G.stabilizer(1)
	T = S.orbits()
	T = list(T)
	T.sort()
	symmetric = []
	asymmetric = []
	all_list = []
	for i in range(1,len(T)):
		O = G.orbit(tuple([1,T[i][0]]),"OnPairs")
		X = DiGraph()
		X.add_edges(O)
		A = X.adjacency_matrix()
		all_list.append(X)
		if A.is_symmetric():
			symmetric.append(Graph(X))
		else:
			asymmetric.append(X)
	asymmetric_pairs = []
	for X in asymmetric:
		A = X.adjacency_matrix()
		for B in asymmetric:
			if (A+B.adjacency_matrix()).is_symmetric() and [B.adjacency_matrix(),A] not in asymmetric_pairs:
				asymmetric_pairs.append([X,B])
	return symmetric,asymmetric_pairs
def sub_orbit_all(G):
	L = sub_orbits(G)
	M = []
	for x in L[1]:
		M += x 
	return L[0],list(set([x.copy(immutable=True) for x in M]))
def is_multiplicity_free(G,H):
	phi = H.trivial_character().induct(G)
	T = phi.decompose()
	for f in T:
		if f[0]>1:
			return False
		else:
			pass 
	return True

In [10]:
def rank_of_group(G):
	return len(G.stabilizer(1).orbits())
def find_non_multiplicity_free(n):
	L = []
	for n in [3..n]:
		for G in PrimitiveGroups(n):
			if G.is_solvable() == False and rank_of_group(G)>6:
				if is_multiplicity_free(G,G.stabilizer(1)) == False:
					L.append((n,G.group_primitive_id()))
					print ((n,G.group_primitive_id()),G)
	return L

In [11]:
find_non_multiplicity_free(153)

(55, 1) PSL(2, 11)
(78, 1) PSL(2, 13)
(91, 1) PSL(2, 13)
(91, 3) PSL(2, 13)
(136, 1) PSL(2, 17)
(153, 1) PSL(2, 17)


[(55, 1), (78, 1), (91, 1), (91, 3), (136, 1), (153, 1)]

In [11]:
def key(T):
	A = T[0].adjacency_matrix()
	for i in [1..len(T)-1]:
		A += T[i].adjacency_matrix()
	return A.is_symmetric()
def check_non_diagonal_general_group(G,k):
	if gap.RankAction(G)>=5:
		T = []#L = sub_orbit_all(G)
		L = sub_orbit_all(G)
		C = Combinations(L[1]+L[0],k)
		print (len(L[0]+L[1]),len(L[0]),len(L[1]))
		for X in C:
			if key(X) == False:
				A = X[0].adjacency_matrix()
				for i in [1..len(X)-1]:
					A += X[i].adjacency_matrix()
					#B = Y.adjacency_matrix()
				C = A
				f = C.minimal_polynomial() 
				if gcd(f,f.derivative()) == 1:
					pass
				else:
					print ("Found")
					T.append([X,A])#return G,X,[Y.degree()[0] for Y in X]
                    
	return T#print ("None")
    

In [12]:
G = PrimitiveGroup(153,1)
r = rank_of_group(G)
#for k in [1..r]:
#    print(check_non_diagonal_general_group(G,k))

In [13]:
T = check_non_diagonal_general_group(G,3)
for x in T:
    L,A = x
    print (A.minimal_polynomial().factor())

14 10 4
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
(x - 40) * (x - 1) * (x + 2)^2 * (x^2 - 52) * (x^3 - 3*x^2 - 33*x + 107)
(x - 40) * (x - 6) * (x - 4) * (x + 2)^2 * (x^2 + 8*x + 20) * (x^3 - 57*x - 163)
(x - 40) * (x + 5) * (x + 10) * (x + 2)^2 * (x^2 - 8*x + 20) * (x^3 - 9*x^2 + 24*x - 19)
(x - 40) * (x - 2) * (x + 2)^2 * (x^3 - 6*x^2 - 72*x + 296)
(x - 40) * (x - 1) * (x + 2)^2 * (x^2 - 52) * (x^3 - 3*x^2 - 33*x + 107)
(x - 40) * (x - 6) * (x - 4) * (x + 2)^2 * (x^2 + 8*x + 20) * (x^3 - 57*x - 163)
(x - 40) * (x + 5) * (x + 10) * (x + 2)^2 * (x^2 - 8*x + 20) * (x^3 - 9*x^2 + 24*x - 19)
(x - 40) * (x - 2) * (x + 2)^2 * (x^3 - 6*x^2 - 72*x + 296)
(x - 32) * (x + 4) * (x - 2)^2 * (x^2 - 68) * (x^3 + 6*x^2 - 9*x - 17)
(x - 32) * (x + 4) * (x - 2)^2 * (x^2 - 68) * (x^3 + 6*x^2 - 9*x - 17)
(x - 32) * (x + 4) * (x - 2)^2 * (x^2 - 68) * (x^3 + 6*x^2 - 9*x - 17)
(x - 32) * (x + 4) * (x - 2)^2 * (x^2 - 68) * (x^3 + 6*x^2 - 9*x - 17)


In [18]:
def label_subsets(n,k):
    L = Combinations([1..n],k)
    D = dict()
    i = 1
    for x in L:
        D[tuple(x)] = i
        i += 1
    return D,L
def action_of_element_as_permutation_on_subsets(k,f,T):
    u = tuple([f(T[i]) for i in [0..k-1]])
    return sorted(u)
def group_acting_on_subsets(G,k):
    PermGens = []
    n = G.degree()
    S = G.gens()
    K = label_subsets(n,k)
    L = K[1]
    D = K[0]
    for g in S:
        N = [0]*binomial(n,k)
        for x in L:
            i = D[tuple(x)]
            j = D[tuple(action_of_element_as_permutation_on_subsets(k,g,x))]
            N[i-1] = j
        PermGens.append(Permutation(N))
    return PermutationGroup(PermGens)
def equitable_partitions_two(q):
	G = PSL(2,q)
	K = group_acting_on_subsets(G,2)
	M = K.conjugacy_classes_subgroups()
	for x in M:
		if x.order() == (q-1)/2 and x.is_abelian() == False:
			H = x 
			print (H.structure_description())
			break
	O = H.orbits()
	L = sub_orbit_all(K)
	print ("Computing minimal polynomials of the equitable partition of the orbits of {0}...\n ".format(H.structure_description()))
	#print ("-----------------------------------------------")
	for x in L[0]:
		for y in L[1]:
			A = x.adjacency_matrix()
			B = y.adjacency_matrix()
			C = A+B
			X = DiGraph(C)
			X.relabel([1..X.order()])
			R = X.is_equitable(O,quotient_matrix=True)
			f = R.minimal_polynomial()
			#print (gcd(f,f.derivative()))
			if gcd(f,f.derivative()) != 1:
				return f.factor()
	return None 

In [20]:
equitable_partitions_two(49)

D12
Computing minimal polynomials of the equitable partition of the orbits of D12...
 


(x - 48) * (x - 14) * x * (x + 2) * (x + 4) * (x - 6)^2 * (x^2 - 6*x - 28) * (x^2 + 3*x - 9) * (x^2 + 10*x + 12) * (x^4 + 12*x^3 + 72*x^2 + 144*x + 144) * (x^10 - 5*x^9 - 70*x^8 + 270*x^7 + 1355*x^6 - 3307*x^5 - 11095*x^4 + 12795*x^3 + 36180*x^2 - 10430*x - 29449)

In [21]:
equitable_partitions_two(25)

D6
Computing minimal polynomials of the equitable partition of the orbits of D6...
 


(x - 36) * (x + 4)^2 * (x^2 - 4*x - 8) * (x^2 - 4*x + 16) * (x^6 - 3*x^5 - 97*x^4 + 238*x^3 + 2358*x^2 - 2783*x - 16589)

In [ ]:
equitable_partitions_two(81)

In [83]:
L = [[((0, 1), (1, 0)), ((1, 0), (0, 1))],
 [((0, 1), (1, 1)),
  ((0, 1), (1, 2)),
  ((0, 1), (1, 4)),
  ((0, 1), (1, 8)),
  ((0, 1), (1, 9)),
  ((0, 1), (1, 13)),
  ((0, 1), (1, 15)),
  ((0, 1), (1, 16)),
  ((1, 0), (1, 1)),
  ((1, 0), (1, 2)),
  ((1, 0), (1, 4)),
  ((1, 0), (1, 8)),
  ((1, 0), (1, 9)),
  ((1, 0), (1, 13)),
  ((1, 0), (1, 15)),
  ((1, 0), (1, 16))],
 [((0, 1), (1, 3)),
  ((0, 1), (1, 5)),
  ((0, 1), (1, 6)),
  ((0, 1), (1, 7)),
  ((0, 1), (1, 10)),
  ((0, 1), (1, 11)),
  ((0, 1), (1, 12)),
  ((0, 1), (1, 14)),
  ((1, 0), (1, 3)),
  ((1, 0), (1, 5)),
  ((1, 0), (1, 6)),
  ((1, 0), (1, 7)),
  ((1, 0), (1, 10)),
  ((1, 0), (1, 11)),
  ((1, 0), (1, 12)),
  ((1, 0), (1, 14))],
 [((1, 1), (0, 1)),
  ((1, 1), (1, 0)),
  ((1, 2), (0, 1)),
  ((1, 2), (1, 0)),
  ((1, 4), (0, 1)),
  ((1, 4), (1, 0)),
  ((1, 8), (0, 1)),
  ((1, 8), (1, 0)),
  ((1, 9), (0, 1)),
  ((1, 9), (1, 0)),
  ((1, 13), (0, 1)),
  ((1, 13), (1, 0)),
  ((1, 15), (0, 1)),
  ((1, 15), (1, 0)),
  ((1, 16), (0, 1)),
  ((1, 16), (1, 0))],
 [((1, 1), (1, 2)),
  ((1, 1), (1, 9)),
  ((1, 2), (1, 1)),
  ((1, 2), (1, 4)),
  ((1, 4), (1, 2)),
  ((1, 4), (1, 8)),
  ((1, 8), (1, 4)),
  ((1, 8), (1, 16)),
  ((1, 9), (1, 1)),
  ((1, 9), (1, 13)),
  ((1, 13), (1, 9)),
  ((1, 13), (1, 15)),
  ((1, 15), (1, 13)),
  ((1, 15), (1, 16)),
  ((1, 16), (1, 8)),
  ((1, 16), (1, 15))],
 [((1, 1), (1, 3)),
  ((1, 1), (1, 6)),
  ((1, 2), (1, 6)),
  ((1, 2), (1, 12)),
  ((1, 4), (1, 7)),
  ((1, 4), (1, 12)),
  ((1, 8), (1, 7)),
  ((1, 8), (1, 14)),
  ((1, 9), (1, 3)),
  ((1, 9), (1, 10)),
  ((1, 13), (1, 5)),
  ((1, 13), (1, 10)),
  ((1, 15), (1, 5)),
  ((1, 15), (1, 11)),
  ((1, 16), (1, 11)),
  ((1, 16), (1, 14))],
 [((1, 1), (1, 4)),
  ((1, 1), (1, 13)),
  ((1, 2), (1, 8)),
  ((1, 2), (1, 9)),
  ((1, 4), (1, 1)),
  ((1, 4), (1, 16)),
  ((1, 8), (1, 2)),
  ((1, 8), (1, 15)),
  ((1, 9), (1, 2)),
  ((1, 9), (1, 15)),
  ((1, 13), (1, 1)),
  ((1, 13), (1, 16)),
  ((1, 15), (1, 8)),
  ((1, 15), (1, 9)),
  ((1, 16), (1, 4)),
  ((1, 16), (1, 13))],
 [((1, 1), (1, 5)),
  ((1, 1), (1, 7)),
  ((1, 2), (1, 10)),
  ((1, 2), (1, 14)),
  ((1, 4), (1, 3)),
  ((1, 4), (1, 11)),
  ((1, 8), (1, 5)),
  ((1, 8), (1, 6)),
  ((1, 9), (1, 11)),
  ((1, 9), (1, 12)),
  ((1, 13), (1, 6)),
  ((1, 13), (1, 14)),
  ((1, 15), (1, 3)),
  ((1, 15), (1, 7)),
  ((1, 16), (1, 10)),
  ((1, 16), (1, 12))],
 [((1, 1), (1, 8)),
  ((1, 1), (1, 15)),
  ((1, 2), (1, 13)),
  ((1, 2), (1, 16)),
  ((1, 4), (1, 9)),
  ((1, 4), (1, 15)),
  ((1, 8), (1, 1)),
  ((1, 8), (1, 13)),
  ((1, 9), (1, 4)),
  ((1, 9), (1, 16)),
  ((1, 13), (1, 2)),
  ((1, 13), (1, 8)),
  ((1, 15), (1, 1)),
  ((1, 15), (1, 4)),
  ((1, 16), (1, 2)),
  ((1, 16), (1, 9))],
 [((1, 1), (1, 10)),
  ((1, 1), (1, 12)),
  ((1, 2), (1, 3)),
  ((1, 2), (1, 7)),
  ((1, 4), (1, 6)),
  ((1, 4), (1, 14)),
  ((1, 8), (1, 11)),
  ((1, 8), (1, 12)),
  ((1, 9), (1, 5)),
  ((1, 9), (1, 6)),
  ((1, 13), (1, 3)),
  ((1, 13), (1, 11)),
  ((1, 15), (1, 10)),
  ((1, 15), (1, 14)),
  ((1, 16), (1, 5)),
  ((1, 16), (1, 7))],
 [((1, 1), (1, 11)),
  ((1, 1), (1, 14)),
  ((1, 2), (1, 5)),
  ((1, 2), (1, 11)),
  ((1, 4), (1, 5)),
  ((1, 4), (1, 10)),
  ((1, 8), (1, 3)),
  ((1, 8), (1, 10)),
  ((1, 9), (1, 7)),
  ((1, 9), (1, 14)),
  ((1, 13), (1, 7)),
  ((1, 13), (1, 12)),
  ((1, 15), (1, 6)),
  ((1, 15), (1, 12)),
  ((1, 16), (1, 3)),
  ((1, 16), (1, 6))],
 [((1, 1), (1, 16)),
  ((1, 2), (1, 15)),
  ((1, 4), (1, 13)),
  ((1, 8), (1, 9)),
  ((1, 9), (1, 8)),
  ((1, 13), (1, 4)),
  ((1, 15), (1, 2)),
  ((1, 16), (1, 1))],
 [((1, 3), (0, 1)),
  ((1, 3), (1, 0)),
  ((1, 5), (0, 1)),
  ((1, 5), (1, 0)),
  ((1, 6), (0, 1)),
  ((1, 6), (1, 0)),
  ((1, 7), (0, 1)),
  ((1, 7), (1, 0)),
  ((1, 10), (0, 1)),
  ((1, 10), (1, 0)),
  ((1, 11), (0, 1)),
  ((1, 11), (1, 0)),
  ((1, 12), (0, 1)),
  ((1, 12), (1, 0)),
  ((1, 14), (0, 1)),
  ((1, 14), (1, 0))],
 [((1, 3), (1, 1)),
  ((1, 3), (1, 9)),
  ((1, 5), (1, 13)),
  ((1, 5), (1, 15)),
  ((1, 6), (1, 1)),
  ((1, 6), (1, 2)),
  ((1, 7), (1, 4)),
  ((1, 7), (1, 8)),
  ((1, 10), (1, 9)),
  ((1, 10), (1, 13)),
  ((1, 11), (1, 15)),
  ((1, 11), (1, 16)),
  ((1, 12), (1, 2)),
  ((1, 12), (1, 4)),
  ((1, 14), (1, 8)),
  ((1, 14), (1, 16))],
 [((1, 3), (1, 2)),
  ((1, 3), (1, 13)),
  ((1, 5), (1, 9)),
  ((1, 5), (1, 16)),
  ((1, 6), (1, 4)),
  ((1, 6), (1, 9)),
  ((1, 7), (1, 2)),
  ((1, 7), (1, 16)),
  ((1, 10), (1, 1)),
  ((1, 10), (1, 15)),
  ((1, 11), (1, 8)),
  ((1, 11), (1, 13)),
  ((1, 12), (1, 1)),
  ((1, 12), (1, 8)),
  ((1, 14), (1, 4)),
  ((1, 14), (1, 15))],
 [((1, 3), (1, 4)),
  ((1, 3), (1, 15)),
  ((1, 5), (1, 1)),
  ((1, 5), (1, 8)),
  ((1, 6), (1, 8)),
  ((1, 6), (1, 13)),
  ((1, 7), (1, 1)),
  ((1, 7), (1, 15)),
  ((1, 10), (1, 2)),
  ((1, 10), (1, 16)),
  ((1, 11), (1, 4)),
  ((1, 11), (1, 9)),
  ((1, 12), (1, 9)),
  ((1, 12), (1, 16)),
  ((1, 14), (1, 2)),
  ((1, 14), (1, 13))],
 [((1, 3), (1, 5)),
  ((1, 3), (1, 12)),
  ((1, 5), (1, 3)),
  ((1, 5), (1, 14)),
  ((1, 6), (1, 7)),
  ((1, 6), (1, 10)),
  ((1, 7), (1, 6)),
  ((1, 7), (1, 11)),
  ((1, 10), (1, 6)),
  ((1, 10), (1, 11)),
  ((1, 11), (1, 7)),
  ((1, 11), (1, 10)),
  ((1, 12), (1, 3)),
  ((1, 12), (1, 14)),
  ((1, 14), (1, 5)),
  ((1, 14), (1, 12))],
 [((1, 3), (1, 6)),
  ((1, 3), (1, 10)),
  ((1, 5), (1, 10)),
  ((1, 5), (1, 11)),
  ((1, 6), (1, 3)),
  ((1, 6), (1, 12)),
  ((1, 7), (1, 12)),
  ((1, 7), (1, 14)),
  ((1, 10), (1, 3)),
  ((1, 10), (1, 5)),
  ((1, 11), (1, 5)),
  ((1, 11), (1, 14)),
  ((1, 12), (1, 6)),
  ((1, 12), (1, 7)),
  ((1, 14), (1, 7)),
  ((1, 14), (1, 11))],
 [((1, 3), (1, 7)),
  ((1, 3), (1, 11)),
  ((1, 5), (1, 6)),
  ((1, 5), (1, 7)),
  ((1, 6), (1, 5)),
  ((1, 6), (1, 14)),
  ((1, 7), (1, 3)),
  ((1, 7), (1, 5)),
  ((1, 10), (1, 12)),
  ((1, 10), (1, 14)),
  ((1, 11), (1, 3)),
  ((1, 11), (1, 12)),
  ((1, 12), (1, 10)),
  ((1, 12), (1, 11)),
  ((1, 14), (1, 6)),
  ((1, 14), (1, 10))],
 [((1, 3), (1, 8)),
  ((1, 3), (1, 16)),
  ((1, 5), (1, 2)),
  ((1, 5), (1, 4)),
  ((1, 6), (1, 15)),
  ((1, 6), (1, 16)),
  ((1, 7), (1, 9)),
  ((1, 7), (1, 13)),
  ((1, 10), (1, 4)),
  ((1, 10), (1, 8)),
  ((1, 11), (1, 1)),
  ((1, 11), (1, 2)),
  ((1, 12), (1, 13)),
  ((1, 12), (1, 15)),
  ((1, 14), (1, 1)),
  ((1, 14), (1, 9))],
 [((1, 3), (1, 14)),
  ((1, 5), (1, 12)),
  ((1, 6), (1, 11)),
  ((1, 7), (1, 10)),
  ((1, 10), (1, 7)),
  ((1, 11), (1, 6)),
  ((1, 12), (1, 5)),
  ((1, 14), (1, 3))]]